In [1]:
import os
import re
import sys
from os import listdir
from os.path import isfile, join
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
from joblib import Parallel, delayed
from PIL import Image
from skimage import exposure, io
from tqdm.notebook import tqdm, trange

In [2]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / "data"

In [3]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils

In [4]:
donors = [
    "LN Donor A",
    "LN Donor E",
    "INT Donor B",
    "INT Donor E",
    "TS Donor A",
    "TS Donor E",
    "SP Donor A"
]

# Import data

In [5]:
import matplotlib.patches as mpatches
from skimage.segmentation import mark_boundaries

def get_imgs(file_path, name):
    f = h5py.File(file_path, "r")
    imgs = f[name]
    labels = list(f[name].attrs["labels"])
    return imgs, labels

def get_img_size(roi_dict, size=1000):
    row_max = 0
    col_max = 0
    for k, v in roi_dict.items():
        row_max = max(row_max, v[0])
        col_max = max(col_max, v[1])
    return row_max + size, col_max + size

def get_img_subset(imgs, markers, labels):
    imgs_subset = []
    for marker in markers:
        idx = labels.index(marker)
        imgs_subset.append(imgs[idx])
    return np.stack(imgs_subset, axis=2)

def get_img_subset1(imgs, markers, labels):
    imgs_subset = []
    for marker in markers:
        idx = labels.index(marker)
        imgs_subset.append(imgs[...,idx])
    return np.stack(imgs_subset, axis=2)

def contrast_streching(img):
    p2, p98 = np.percentile(img, (0.5, 99.5))
    img = exposure.rescale_intensity(img, in_range=(p2, p98), out_range=(0, 255)).astype(np.uint8)
    return img

In [6]:
# Format row, col
arrangement = {
    "LN Donor A": {
        1: [0, 1000],
        2: [0, 2000],
        3: [1000, 0],
        4: [1000, 1000],
        5: [1000, 2000],
        6: [1000, 3000],
        7: [2000, 0],
        8: [2000, 1000],
        9: [2000, 2000],
        10: [2000, 3000],
        11: [3000, 0],
        12: [3000, 1000],
        13: [3000, 2000],
        14: [3000, 3000],
        15: [4000, 1000],
        16: [4000, 2000],
    },
    "LN Donor E": {
        1: [1000, 0],
        2: [1000, 1000],
        3: [1000, 2000],
        4: [1000, 3000],
        5: [1000, 4000],
        6: [1000, 5000],
        7: [1000, 6000],
        8: [1000, 7000],
        9: [1000, 8000],
        10: [0, 0],
        11: [0, 1000],
        12: [0, 2000],
        13: [0, 3000],
        14: [0, 4000],
    },
    "INT Donor B": {
        1: [0, 0],
        2: [0, 1000],
        3: [1000, 0],
        4: [1000, 1000],
        5: [2000, 0],
        6: [2000, 1000],
        7: [2000, 2000],
        8: [2000, 3000],
        9: [3000, 0],
        10: [3000, 1000],
        11: [3000, 2000],
        12: [3000, 3000],
        13: [4000, 0],
        14: [4000, 1000],
        15: [4000, 2000],
        16: [4000, 3000],
        17: [5000, 0],
        18: [5000, 1000],
        19: [5000, 2000],
        20: [5000, 3000],
    },
    "INT Donor E": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        # 5: [0, 4000],
        6: [1000, 0],
        7: [1000, 1000],
        8: [1000, 2000],
        9: [1000, 3000],
        10: [1000, 4000],
        11: [2000, 3000],
        12: [2000, 4000],
        13: [3000, 3000],
        14: [3000, 4000],
        15: [4000, 3000],
        16: [4000, 4000],
    },
    "TS Donor A": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        5: [0, 4000],
        6: [0, 5000],
        7: [0, 6000],
        8: [1000, 0],
        9: [1000, 1000],
        10: [1000, 2000],
        11: [1000, 3000],
        12: [1000, 4000],
        13: [1000, 5000],
        14: [1000, 6000],
    },
    "TS Donor E": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [1000, 0],
        5: [1000, 1000],
        6: [1000, 2000],
        7: [2000, 0],
        8: [2000, 1000],
        9: [2000, 2000],
        10: [3000, 0],
        11: [3000, 1000],
        12: [3000, 2000],
        13: [4000, 0],
        14: [4000, 1000],
        15: [4000, 2000],
        16: [5000, 0],
        17: [5000, 1000],
        18: [5000, 2000],
    },
    "SP Donor A": {
        1: [0, 0],
        2: [0, 1000],
        3: [0, 2000],
        4: [0, 3000],
        5: [0, 4000],
        6: [1000, 0],
        7: [1000, 1000],
        8: [1000, 2000],
        9: [1000, 3000],
        10: [1000, 4000],
        11: [2000, 0],
        12: [2000, 1000],
        13: [2000, 2000],
        14: [2000, 3000],
        15: [2000, 4000],
        16: [3000, 0],
        17: [3000, 1000],
        18: [3000, 2000],
        19: [3000, 3000],
        20: [3000, 4000],
    }
}

In [12]:
markers = [
    "CD38",
    "Vimentin",
    "CD21",
    "BCL6",
    "ICOS1",
    "CD11b",
    "CD11c",
    "FoxP3",
    "CD4",
    "CD138",
    "CXCR5",
    "CD20",
    "CD8",
    "C-Myc",
    "PD1",
    "CD83",
    "Ki67",
    "COL1",
    "CD3",
    "CD27",
    "EZH2",
]
size = 1000


RGB_MAP = {
    1: {"rgb": np.array([255, 0, 0]), "range": [0, 255]},
    2: {"rgb": np.array([0, 255, 0]), "range": [0, 255]},
    3: {"rgb": np.array([0, 0, 255]), "range": [0, 255]},
    4: {"rgb": np.array([0, 255, 255]), "range": [0, 255]},
    5: {"rgb": np.array([255, 0, 255]), "range": [0, 255]},
    6: {"rgb": np.array([255, 255, 0]), "range": [0, 255]},
    7: {"rgb": np.array([255, 255, 255]), "range": [0, 255]},
}

In [21]:
import tifffile as tiff

In [25]:
# Export combined tiff image 
for donor in donors:
    h5_data = p_dir / "data" / "h5_new" / f"{donor}.hdf5"

    imgs, labels = get_imgs(h5_data, str(1))
    
    
    # Create combined images
    y_max, x_max = get_img_size(arrangement[donor])
    img_combined = np.zeros((len(imgs), y_max, x_max), dtype=np.uint8)

    ROIs = [i for i in range(1, 21)]
    for roi in tqdm(ROIs):
        if roi not in arrangement[donor].keys():
            continue

        # Read imgs
        imgs, labels = get_imgs(h5_data, str(roi))

        # Insert Combined images
        y = arrangement[donor][roi][0]
        x = arrangement[donor][roi][1]
        img_combined[:, y : y + size, x : x + size] = imgs[:,:1000,:1000]
    
    for i in range(img_combined.shape[0]):
        img_combined[i,...] = contrast_streching(img_combined[i,...])
        
    save_path = (
        p_dir / "data" / "whole_stitched" / f"{donor}.tiff"
    )
    # img_combined = Image.fromarray(img_combined)
    # img_combined.save(save_path)
    tiff.imsave(save_path, img_combined, metadata={"axes": "CYX", "Labels": labels}, imagej=True)

  0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\thu71\AppData\Local\Temp\ipykernel_23560\174003330.py:33: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(save_path, img_combined, metadata={"axes": "CYX", "Labels": labels}, imagej=True)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
imgs_donors = {}

for donor in donors:
    h5_data = p_dir / "data" / "h5_new" / f"{donor}.hdf5"

    # Create combined images
    y_max, x_max = get_img_size(arrangement[donor])
    img_combined = np.zeros((y_max, x_max, len(markers)), dtype=np.uint8)

    ROIs = [i for i in range(1, 21)]
    for roi in tqdm(ROIs):
        if roi not in arrangement[donor].keys():
            continue

        # Read imgs
        imgs, labels = get_imgs(h5_data, str(roi))

        # Get multiplex image
        data = get_img_subset(imgs, markers, labels)

        # Insert Combined images
        y = arrangement[donor][roi][0]
        x = arrangement[donor][roi][1]
        img_combined[y : y + size, x : x + size, :] = data[:1000,:1000]

    for i in range(img_combined.shape[2]):
        img_combined[...,i] = contrast_streching(img_combined[...,i])
        
    imgs_donors[donor] = img_combined

  0%|          | 0/20 [00:00<?, ?it/s]

TypeError: Cannot handle this data type: (1, 1, 21), |u1

In [14]:
markers_subset = ["CD3", "CD4", "ICOS1", "CXCR5", "BCL6"]

channels = [i + 1 for i in range(len(markers_subset))]
for donor in donors:
    imgs_all = imgs_donors[donor]
    img_temp = get_img_subset1(imgs_all, markers_subset, markers) 
    img_combined = my_utils.convert_to_rgb(
        img_temp, channels=channels, vmax=255, rgb_map=RGB_MAP
    )
        
    save_path = (
        p_dir / "figures" / "multiplex" / f"combined_{donor}_{markers_subset}.png"
    )
    img_combined = Image.fromarray(img_combined)
    img_combined.save(save_path)

In [15]:
markers_subset = ["CD38", "CD138", "CD27", "Vimentin", "COL1"]
channels = [i + 1 for i in range(len(markers_subset))]
for donor in donors:
    imgs_all = imgs_donors[donor]
    img_temp = get_img_subset1(imgs_all, markers_subset, markers) 
    img_combined = my_utils.convert_to_rgb(
        img_temp, channels=channels, vmax=255, rgb_map=RGB_MAP
    )
        
    save_path = (
        p_dir / "figures" / "multiplex" / f"combined_{donor}_{markers_subset}.png"
    )
    img_combined = Image.fromarray(img_combined)
    img_combined.save(save_path)

In [16]:
markers_subset = ["Ki67", "CD21", "CD20", "C-Myc", "CD83"]
channels = [i + 1 for i in range(len(markers_subset))]
for donor in donors:
    imgs_all = imgs_donors[donor]
    img_temp = get_img_subset1(imgs_all, markers_subset, markers) 
    img_combined = my_utils.convert_to_rgb(
        img_temp, channels=channels, vmax=255, rgb_map=RGB_MAP
    )
        
    save_path = (
        p_dir / "figures" / "multiplex" / f"combined_{donor}_{markers_subset}.png"
    )
    img_combined = Image.fromarray(img_combined)
    img_combined.save(save_path)